In [2]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
import metis
import networkit as nk
from networkit import vizbridges
import pickle
# import powerlaw

/var/folders/lk/t243gzpx379bzd9mvx8dcsmm0000gp/T/ipykernel_8353/3872114279.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# 파일 불러오기
with open('../../data/df.pkl', 'rb') as f:
    df = pickle.load(f)


In [4]:
# 파일 불러오기
with open('../../data/df_qna.pkl', 'rb') as f:
    df_qna = pickle.load(f)

In [5]:
df['q_creationdate'].min()

Timestamp('2023-08-03 01:04:49.693000')

In [6]:
df_gephi = df[['q_owneruserid', 'a_owneruserid']]

In [7]:
df_gephi.columns = ['Source', 'Target']

In [8]:
node = np.unique(np.append(df_gephi['Source'].values, df_gephi['Target'].values))

In [9]:
# 질문자->답변자 로 edgelist 생성
edgelist = list(zip(df_gephi['Source'], df_gephi['Target']))

In [10]:
di_github = nx.DiGraph()
di_github.add_nodes_from(node)
di_github.add_edges_from(edgelist)

github = nx.from_pandas_edgelist(df,  source='q_owneruserid', target='a_owneruserid')

In [11]:
di_github_nk = nk.nxadapter.nx2nk(di_github)
github_nk = nk.nxadapter.nx2nk(github)

In [12]:
print(di_github_nk.numberOfNodes(), di_github_nk.numberOfEdges())

11392 11996


### Core Decomposition

In [13]:
rm_github_nk = github_nk.removeSelfLoops()

In [14]:
type(rm_github_nk)

NoneType

In [15]:
coreDec = nk.centrality.CoreDecomposition(github_nk)
coreDec.run()
# direct 라 안되는데
# untimeError: Core Decomposition implementation does not support graphs with self-loops. Call Graph.removeSelfLoops() first.

In [16]:
# nk.gephi.pyclient.GephiClient(url='http://116.45.16.200:5999/workspace0', autoflush=False)
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:5999/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace1')
# gsc = nk.gephi.streaming.GephiStreamingClient(url='http://143.248.106.43:7777/workspace2')
gsc = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:7777/workspace1')


In [17]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://localhost:8080/workspace1')

In [18]:
# test = nk.gephi.streaming.GephiStreamingClient(url='http://116.45.16.200:8080/workspace2')

In [19]:
# test.exportGraph(di_github_nk)

In [20]:
gsc.exportGraph(di_github_nk)

Could not connect to the gephi streaming plugin.
Did you start the streaming master server in gephi and provide the name of your workspace?
If the workspace is named 'Workspace 0', the corresponding url is http://localhost:8080/workspace0 (adapt port)


In [21]:
# nk.vizbridges.widgetFromGraph(github_nk, dimension = nk.vizbridges.Dimension.Two, nodeScores = coreDec.scores())

In [22]:
di_github_nk
# 1762

In [23]:
list(di_github.nodes())[1762]

2376954

In [24]:
list(di_github.nodes())[5115]

10542284

In [25]:
df[(df['q_owneruserid']==19556938)|(df['a_owneruserid']==19556938)]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
5766,77231004,1,NaN,None,2023-10-04 15:39:13.480,1,51,19556938,Java - Quarkus framework. Panache MongoDB. How...,java mongodb quarkus quarkus-panache,...,77231004,2023-10-04 16:42:50.920,0,None,19556938,None,None,None,0,11


In [26]:
df[(df['q_owneruserid']==list(di_github.nodes())[5115])|(df['a_owneruserid']==list(di_github.nodes())[5115])]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
2667,77410093,1,NaN,None,2023-11-02 13:43:22.463,1,49,10542284,Why Spring Boot Backend is not letting me Logi...,reactjs spring spring-boot,...,77410093,2023-11-04 13:34:54.593,1,None,10542284,None,None,None,0,375
5094,77273036,1,NaN,None,2023-10-11 12:23:58.780,1,54,10542284,SpringBoot GetMapping Throwing always a 404 code,java spring-boot,...,77273036,2023-10-11 12:39:27.233,1,None,19984776,None,None,None,1,11
11474,76868443,1,76877800.0,None,2023-08-09 13:53:10.813,1,55,10542284,I'm getting RESULT_CANCELED from startActivity...,java android kotlin default,...,76868443,2023-08-10 16:50:21.053,1,None,10542284,None,None,None,0,375


In [27]:
df[(df['q_owneruserid']==list(di_github.nodes())[6829])|(df['a_owneruserid']==list(di_github.nodes())[6829])]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
9183,77022398,1,NaN,None,2023-09-01 10:36:23.383,0,795,9849018,Cannot deserialize value of type `java.lang.St...,java deserialization jackson-databind,...,77022398,2023-09-01 11:22:27.080,0,None,15328090,None,None,None,0,91
9186,77021421,1,NaN,None,2023-09-01 08:00:24.553,0,60,12571132,Declare multiple property variables in log4j2....,java xml spring yaml log4j2,...,77021421,2023-09-01 09:55:19.497,0,None,15328090,None,None,None,1,91
9455,77004940,1,NaN,None,2023-08-30 04:10:20.370,0,80,4737012,Java Servlet HTTP Status 405 Method Not Allowed,java spring jsp tomcat java-server,...,77004940,2023-08-30 11:14:31.793,2,None,15328090,None,None,None,2,91


In [28]:
df[(df['q_owneruserid']==list(di_github.nodes())[8907])|(df['a_owneruserid']==list(di_github.nodes())[8907])]

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_parentid,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation
8745,77054214,1,NaN,None,2023-09-06 17:48:55.400,0,41,21674811,Is there annotation for mark a visibility vari...,java android kotlin view annotations,...,77054214,2023-09-06 18:07:59.287,1,None,3621353,None,None,None,2,3263
11247,76891311,1,76891377.0,None,2023-08-12 22:10:42.470,0,44,21674811,Why layout editor in android studio working in...,java android xml kotlin layout,...,76891311,2023-08-12 22:42:53.357,1,None,4592885,None,None,None,0,344


In [31]:
list(di_github.nodes())[8907]

21674811